# Cleanup Test Data

**Purpose:** Reset the environment for testing `job_cdm_dta_import`.

**What it does:**
1. Drops all related tables (bronze, silver, gold)
2. Deletes checkpoint folder from Volumes
3. Deletes ingested folder from historical_data

**⚠️ WARNING:** This will delete all data. Only run in dev/test environments!


In [ ]:
# Cell 1: Configuration
print("="*80)
print("CLEANUP TEST DATA")
print("="*80)

# Get catalog from job parameter - never hardcode
# The job passes catalog_name from databricks.yml test_catalog variable
try:
    CATALOG = dbutils.widgets.get("catalog_name")
    if not CATALOG or not CATALOG.strip():
        raise ValueError("catalog_name parameter is empty")
except Exception as e:
    raise RuntimeError(f"catalog_name parameter required. Pass from job using test_catalog variable. Error: {e}")

# Schema defaults
BRONZE_SCHEMA = "bronze_md"
SILVER_SCHEMA = "silver_md"
GOLD_SCHEMA = "gold_md"

# Volume paths
VOLUME_BASE = f"/Volumes/{CATALOG}/{BRONZE_SCHEMA}/clinical_data_standards"
CHECKPOINT_FOLDER = f"{VOLUME_BASE}/jobs/_checkpoints"
HISTORICAL_INGESTED_FOLDER = f"{VOLUME_BASE}/historical_data/ingested"
TEST_INGESTED_FOLDER = f"{VOLUME_BASE}/test/ingested"

print(f"Catalog: {CATALOG}")
print(f"Bronze Schema: {BRONZE_SCHEMA}")
print(f"Silver Schema: {SILVER_SCHEMA}")
print(f"Gold Schema: {GOLD_SCHEMA}")
print(f"Checkpoint Folder: {CHECKPOINT_FOLDER}")
print(f"Historical Ingested Folder: {HISTORICAL_INGESTED_FOLDER}")
print(f"Test Ingested Folder: {TEST_INGESTED_FOLDER}")


In [ ]:
# Cell 2: Drop Bronze Tables
print(f"\n{'='*60}")
print("Dropping Bronze Tables...")
print(f"{'='*60}")

bronze_tables = [
    "md_dta_history",
    "md_dta_excel_file_raw",
    "md_document_status"
]

for table in bronze_tables:
    full_name = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_name}")
        print(f"  ✓ Dropped: {full_name}")
    except Exception as e:
        print(f"  ✗ Failed to drop {full_name}: {e}")


In [ ]:
# Cell 3: Drop Silver Tables
print(f"\n{'='*60}")
print("Dropping Silver Tables...")
print(f"{'='*60}")

silver_tables = [
    "md_codelists_normalized",
    "md_transfer_variable_field_normalized"
]

for table in silver_tables:
    full_name = f"{CATALOG}.{SILVER_SCHEMA}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_name}")
        print(f"  ✓ Dropped: {full_name}")
    except Exception as e:
        print(f"  ✗ Failed to drop {full_name}: {e}")


In [ ]:
# Cell 4: Drop Gold Tables
print(f"\n{'='*60}")
print("Dropping Gold Tables...")
print(f"{'='*60}")

gold_tables = [
    "md_transfer_variables_library",
    "dta_workflow",
    "dta",
    "md_version_registry"
]

for table in gold_tables:
    full_name = f"{CATALOG}.{GOLD_SCHEMA}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_name}")
        print(f"  ✓ Dropped: {full_name}")
    except Exception as e:
        print(f"  ✗ Failed to drop {full_name}: {e}")


In [ ]:
# Cell 5: Delete Checkpoint Folder
print(f"\n{'='*60}")
print("Deleting Checkpoint Folder...")
print(f"{'='*60}")

try:
    # Check if folder exists
    files = dbutils.fs.ls(CHECKPOINT_FOLDER)
    print(f"  Found {len(files)} items in checkpoint folder")
    
    # Delete recursively
    dbutils.fs.rm(CHECKPOINT_FOLDER, recurse=True)
    print(f"  ✓ Deleted: {CHECKPOINT_FOLDER}")
except Exception as e:
    if "FileNotFoundException" in str(e) or "does not exist" in str(e).lower():
        print(f"  ⏭️  Folder does not exist (OK): {CHECKPOINT_FOLDER}")
    else:
        print(f"  ✗ Failed to delete checkpoint folder: {e}")


In [ ]:
# Cell 6: Delete Ingested Folders
print(f"\n{'='*60}")
print("Deleting Ingested Folders...")
print(f"{'='*60}")

# Delete historical_data/ingested folder
print("\n1. Historical Data Ingested Folder:")
try:
    files = dbutils.fs.ls(HISTORICAL_INGESTED_FOLDER)
    print(f"  Found {len(files)} items")
    dbutils.fs.rm(HISTORICAL_INGESTED_FOLDER, recurse=True)
    print(f"  ✓ Deleted: {HISTORICAL_INGESTED_FOLDER}")
except Exception as e:
    if "FileNotFoundException" in str(e) or "does not exist" in str(e).lower():
        print(f"  ⏭️  Folder does not exist (OK)")
    else:
        print(f"  ✗ Failed to delete: {e}")

# Delete test/ingested folder
print("\n2. Test Data Ingested Folder:")
try:
    files = dbutils.fs.ls(TEST_INGESTED_FOLDER)
    print(f"  Found {len(files)} items")
    dbutils.fs.rm(TEST_INGESTED_FOLDER, recurse=True)
    print(f"  ✓ Deleted: {TEST_INGESTED_FOLDER}")
except Exception as e:
    if "FileNotFoundException" in str(e) or "does not exist" in str(e).lower():
        print(f"  ⏭️  Folder does not exist (OK)")
    else:
        print(f"  ✗ Failed to delete: {e}")


In [ ]:
# Cell 7: Summary
print(f"\n{'='*80}")
print("CLEANUP COMPLETE")
print(f"{'='*80}")

print(f"""
Summary:
  Catalog: {CATALOG}
  Bronze tables dropped: {len(bronze_tables)}
  Silver tables dropped: {len(silver_tables)}
  Gold tables dropped: {len(gold_tables)}
  
  Folders cleaned:
    - {CHECKPOINT_FOLDER}
    - {HISTORICAL_INGESTED_FOLDER}
    - {TEST_INGESTED_FOLDER}

Next steps:
  1. Upload test data to: {VOLUME_BASE}/test/uploads/
  2. Run job_cdm_dta_import_test to process the data
""")
